In [1]:
from scipy.signal import coherence as msc
from scipy.signal import welch as psdw
from scipy import signal as sgl
import numpy as np
from lib.stats import *
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from lib.matplotlib2tikz import save as tikz_save

In [2]:
x=np.loadtxt('spectralData/xi50pt007.dat',skiprows=0,unpack=True,usecols=range(102))
xcl=np.loadtxt('spectralData/clSignal.dat',skiprows=0,unpack=True,usecols=range(4))

In [3]:
dataset='signalData/intp1.dat'
xcp=np.loadtxt(dataset,skiprows=0,usecols=range(2),unpack=True)
dataset='signalData/intp2.dat'
xcp2=np.loadtxt(dataset,skiprows=0,usecols=range(2),unpack=True)

In [4]:
n=4000;n2=n
s1=x.shape;s2=xcl.shape;s3=xcp.shape
lzee=x.shape[0];lzes=1;lze=lzee-lzes
for i in range(lzes,lzee):
    x[i,:]=x[i,:]-x[i,:].mean()
    if (i==lzes):
        out1,out2,nsam,fsam=rsample(x[i,:],x[0,:],n,verbose=True)
        z=np.zeros((s1[0],len(out1)))
        z[i,:]=out1[:];z[0,:]=out2[:]
    else:
        z[i,:],out2,nsam,fsam=rsample(x[i,:],x[0,:],n)

for i in range(2,4):
    xcl[i,:]=xcl[i,:]-xcl[i,:].mean()
    if (i==2):
        out1,out2,nsam,fsam=rsample(xcl[i,:],xcl[1,:],n2,verbose=True)
        zcl=np.zeros((s2[0],len(out1)))
        zcl[i,:]=out1[:];zcl[1,:]=out2[:]
    else:
        zcl[i,:],out2,nsam,fsam=rsample(xcl[i,:],xcl[1,:],n2)

for i in [1]:
    xcp[i,:]=xcp[i,:]-xcp[i,:].mean()
    if (i==1):
        out1,out2,nsam,fsam=rsample(xcp[i,:],xcp[0,:],n2,verbose=True)
        zcp=np.zeros((s3[0],len(out1)))
        zcp[i,:]=out1[:];zcp[0,:]=out2[:]
for i in [1]:
    xcp2[i,:]=xcp2[i,:]-xcp2[i,:].mean()
    if (i==1):
        out1,out2,nsam,fsam=rsample(xcp2[i,:],xcp2[0,:],n2,verbose=True)
        zcp2=np.zeros((s3[0],len(out1)))
        zcp2[i,:]=out1[:];zcp2[0,:]=out2[:]

tsam=1/fsam; myf=1

# of samples modified to:  4096
fmax: 68.250455003  fmin: 0.0333254174819
# of samples modified to:  4096
fmax: 68.250364002  fmin: 0.0333253730478
# of samples modified to:  4096
fmax: 68.250455003  fmin: 0.0333254174819
# of samples modified to:  4096
fmax: 68.250455003  fmin: 0.0333254174819


In [8]:
plot=1
ovlp=0.0;nwin=10;


ntotal=z.shape[1];lzee=z.shape[0];lzes=1;lze=lzee-lzes
ntt=0
start=[];end=[]
ofst=1.0-ovlp
nt=ntotal-1
nseg=round(float(nt)/((nwin-1)*ofst+1.0));print('nseg0=',nseg)
while(ntt!=ntotal):
    if(nwin==1):
        ofst=1.0
    else:
        ofst=(float(nt)/nseg-1)/(nwin-1)
    nseg=round(float(nt)/((nwin-1)*ofst+1.0));
    if(nwin==1):
        ofst=1.0
    else:
        ofst=(float(nt)/nseg-1)/(nwin-1);
    ovlp=1.0-ofst
    iovlp=int(np.ceil(ovlp*nseg));iofst=nseg-iovlp
    
    flag=0
    while (ofst>1.0):
        nseg+=1
        ofst=(float(nt)/nseg)/(nwin-1)
        flag=1
    if(flag==1):
        print('Offset was more than segment length!!')
    ovlp=1.0-ofst 

    print('Overlap modified to ',round(ovlp,3),'to fit data')
    iovlp=int(np.ceil(ovlp*nseg));iofst=nseg-iovlp

    for i in range(nwin):
        start.append((i)*iofst)
        end.append((i)*iofst+nseg)
    ntt=1+end[-1]-start[0]
    if(ntt!=ntotal):
        nseg+=1

if(plot==1):  
    ax1=plt.subplot(3,1,1)
    ctitle=('ntotal='+str(ntt)+' nseg='+str(nseg)+' iovlp='+str(iovlp))
    ctitle2=('\n Final overlap of segments is '+str(round(100.0*ovlp,3))+'%')
    plt.title(ctitle+ctitle2)
    plt.ylabel('psurf')
    ax2=plt.subplot(3,1,3,sharex=ax1)
    plt.ylabel('Cl')
    ax3=plt.subplot(3,1,2,sharex=ax1)
    plt.ylabel('pshed')
    plt.xlabel('t')
    for i in range(nwin):
        ax1.plot(z[0,start[i]:end[i]+1],z[37,start[i]:end[i]+1],label='s'+str(i+1))
        ax2.plot(z[0,start[i]:end[i]+1],zcl[2,start[i]:end[i]+1])
        ax3.plot(z[0,start[i]:end[i]+1],zcp[1,start[i]:end[i]+1])

    #ax1.legend(bbox_to_anchor=(0., -0.115, 1., -.15), loc=3,
    #           ncol=nwin, mode="expand", borderaxespad=0.)
    
    plt.show()
fmax=fsam/2; fmin=1/(z[0,end[0]]-z[0,start[0]])
print('fmax:',fmax,' fmin:',fmin)

nseg0= 410
Overlap modified to  0.001 to fit data
Overlap modified to  0.004 to fit data
Overlap modified to  0.007 to fit data
Overlap modified to  0.009 to fit data
Overlap modified to  0.012 to fit data
fmax: 68.250455003  fmin: 0.332929048795


In [7]:
fmax=10
for i in range(lzes,lzee):
    ff,clp=msc(zcl[2,:],z[i,:],fs=fsam,window='hann',nperseg=nseg,noverlap=iovlp)
    if (i==lzes):
        Clp=np.zeros((lze,ff.shape[0]))
    Clp[i-1,:]=clp[:]
zz=range(101)
X, Y = np.meshgrid(ff,zz)
plt.contourf(X,Y,Clp[:,:])
plt.ylabel(r'$\zeta$',fontsize=16)
plt.xlabel(r'$f^*$',fontsize=16)
plt.xlim(0,fmax)
plt.colorbar();
plt.show()

/home/rpt1g12/anaconda3/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [10]:
fmax=15; psd=1
for i in range(lzes,lzee):
    if (psd==0):
        v = np.linspace(0.0, 1.0, 11, endpoint=True)
        ff,cpp=msc(zcp[1,:],z[i,:],fs=fsam,
                   window='hann',nperseg=nseg,noverlap=iovlp)
    else:
        ff,cpp=psdw(z[i,:],fs=fsam,
                    window='hann',nperseg=nseg,noverlap=iovlp)
    if (i==lzes):
        Cpp=np.zeros((lze,ff.shape[0]))
    Cpp[i-1,:]=cpp[:]
zz=[i*(2.0/100) for i in range(101)]
X, Y = np.meshgrid(ff,zz)
if (psd==1):
    CS=plt.contourf(X,Y,Cpp[:,:])#,locator=ticker.LogLocator())
    def fmt(x, pos):
        a, b = '{:.2e}'.format(x).split('e')
        b = int(b)
        return r'${} \times 10^{{{}}}$'.format(a, b)

    cbar=plt.colorbar(CS, format=ticker.FuncFormatter(fmt))
    cbar.ax.set_ylabel('PSD')
else:
    CS=plt.contourf(X,Y,Cpp[:,:],v)
    cbar=plt.colorbar(CS)
    cbar.ax.set_ylabel('MSC')
plt.ylabel(r'$\frac{z}{L_c \lambda_{LE}}$',fontsize=25)
plt.xlabel(r'$f^*$',fontsize=20)
plt.xlim(0,fmax)
plt.show()

/home/rpt1g12/anaconda3/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [21]:
fmax=15
ff5,pcp=psdw(zcp[1,:],fs=fsam,nperseg=nseg,noverlap=iovlp)
ff52,pcp2=psdw(zcp2[1,:],fs=fsam,nperseg=nseg,noverlap=iovlp)
plt.plot(ff5,pcp,'b-o',label=r'$p_{T1}$')
plt.plot(ff52,pcp2,'g-s',label=r'$p_{T2}$')
plt.xlabel(r'$f^*$')
plt.ylabel(r'PSD')
plt.ylim(1.0e-8,1.0e-4)
#plt.title('xi[58-63]eta[40-48]zeta[85-90]')
#plt.title('xi[52-58]eta[45-60]zeta[35-39]')
plt.grid(b=True, which='major', color='k', linestyle=':')
plt.grid(b=True, which='minor', color='k', linestyle=':')
plt.xlim(0,fmax)
plt.legend(ncol=1, borderaxespad=0.3)
tikzpath='/home/rpt1g12/Dropbox/phd/figures/wleResults/'
tikz_save(tikzpath+'cwlepsdpT12.tex',
          figurewidth = '\\tikzw',
          figureheight = '\\tikzh')
plt.show()

file encoding: UTF-8
Problem during transformation, continuing with original data
Problem during transformation, continuing with original data
Problem during transformation, continuing with original data
Problem during transformation, continuing with original data
Problem during transformation, continuing with original data
Problem during transformation, continuing with original data
Problem during transformation, continuing with original data
Problem during transformation, continuing with original data
Problem during transformation, continuing with original data
Problem during transformation, continuing with original data
Please add the following line to your LaTeX preamble:

\usepackage{pgfplots}
\usepgfplotslibrary{plotmarks}


In [24]:
m=37
plt.plot(ff[:],Clp[m,:],'-o')
plt.xlim(0,fmax)
plt.ylabel('msc')
plt.xlabel('f*')
plt.show()

In [24]:
for j in range(lzes,lzee):
    for i in range(lzes,lzee):
        ff,cpp=msc(z[j,0:ntt],z[i,0:ntt],fs=fsam,nperseg=nseg,noverlap=iovlp)
        if (i==lzes and j==lzes):
            Cpp=np.zeros((lze,lze,ff.shape[0]))
        Cpp[j-1,i-1,:]=cpp
v = np.linspace(0.0, 1.0, 11, endpoint=True)
zz=[i*(2.0/100) for i in range(101)]
X, Y = np.meshgrid(ff, zz)

In [41]:
i=7;fmax=15
m=int(round(12.5*i))
if (i%2==0):
    countm=i//2+1
    cs="Middle "
    cs+=str(countm)
else:
    if((i//2)%2==0):
        countp=i//4+1
        cs="Peak "
        cs+=str(countp)
    else:
        countt=i//4+1
        cs="Trough "
        cs+=str(countt)
CS=plt.contourf(X[:,:],Y[:,:],Cpp[m,:,:],v)
plt.ylabel(r'$\frac{z}{L_c \lambda_{LE}}$',fontsize=25)
plt.xlabel(r'$f^*$',fontsize=16)
plt.title(cs+' used as base Signal')
cbar=plt.colorbar(CS)
cbar.ax.set_ylabel('MSC')
plt.xlim(0,fmax)
plt.show()

/home/rpt1g12/anaconda3/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [14]:
m=round(12.5*0)+1
ff,CC=psdw(zcl[2,:],fs=fsam,nperseg=nseg,noverlap=iovlp)
plt.plot(ff[0:fmax],CC[0:fmax]);plt.show()

In [1]:
for i in range(9):
    m=int(round(12.5*i))
    if (i%2==0): 
        cS='M'
    else:
        if ((i//2)%2==0):
            cS='P'
        else:
            cS='T'
    print(str(m)+cS)

0M
12P
25M
38T
50M
62P
75M
88T
100M
